## Train

In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Bijay\\Downloads\\DJANGO_PROJECTS\\ML\\dl_project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Bijay\\Downloads\\DJANGO_PROJECTS\\ML\\dl_project'

## Entity

In [45]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainConfig:
    root_dir: Path
    train_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epoch: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

## Configuration Manager

In [46]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_dir
import tensorflow as tf

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dir([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        
        config = self.config.prepare_callback_model
        model_ckpt_model = os.path.dirname(config.checkpoint_model_filepath)
        create_dir([
            Path(model_ckpt_model),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    
    def get_training_config(self) -> TrainConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")
        create_dir([
            Path(training.root_dir)
        ])

        training_config = TrainConfig(
            root_dir = Path(training.root_dir),
            train_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epoch = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
        )

        return training_config

In [47]:
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property # with it we can access the method like a variable
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_log_dir_at {timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )

    def get_tb_chpt_callback(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [ ]:
import os
from pathlib import Path
import tensorflow as tf

class Training:
    def __init__(self, config: TrainConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
        self.config.updated_base_model_path,
        compile=False
    )
    
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss='binary_crossentropy',  
            metrics=['accuracy']
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255, #normalizing from [0, 255] to [0, 1]
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epoch,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.train_model_path,
            model=self.model
        )

In [51]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallback(config=prepare_callback_config)
    callback_list = prepare_callback.get_tb_chpt_callback()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-09-14 16:02:03,308: INFO: common_cmd: Yaml file : config\config.yaml loaded successfully]
[2025-09-14 16:02:03,312: INFO: common_cmd: Yaml file : params.yaml loaded successfully]
[2025-09-14 16:02:03,314: INFO: common_cmd: Directory created succesfully at artifacts]
[2025-09-14 16:02:03,315: INFO: common_cmd: Directory created succesfully at artifacts\prepare_callback\checkpoint_dir]
[2025-09-14 16:02:03,317: INFO: common_cmd: Directory created succesfully at artifacts\prepare_callback\tensorboard_log_dir]
[2025-09-14 16:02:03,319: INFO: common_cmd: Directory created succesfully at artifacts\training]
Found 78 images belonging to 2 classes.
Found 318 images belonging to 2 classes.
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.5255 - loss: 4.6130[2025-09-14 16:03:43,746: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras forma

- tensorboard --logdir=artifacts/prepare_callback/tensorboard_log_dir
- to open the tensorboard 